In [78]:
%load_ext autoreload
%autoreload 2
import logging
logging.basicConfig(level=logging.INFO)
import random
random.seed(40)

INFO:root:Creating initial trades
INFO:root:Adding trade with UID: 1
INFO:root:Adding trade with UID: 2
INFO:root:Adding trade with UID: 3
INFO:root:Adding trade with UID: 4
INFO:root:Adding more trades to the journal
INFO:root:Adding trade with UID: 5
INFO:root:Adding trade with UID: 6
INFO:root:Adding trade with UID: 7
INFO:root:Adding trade with UID: 8


INFO:root:Adding trade with UID: 9
INFO:root:Adding trade with UID: 10
INFO:root:Adding trade with UID: 11
INFO:root:Adding trade with UID: 12
INFO:root:Adding trade with UID: 13
INFO:root:Adding trade with UID: 14
INFO:root:Adding default tags to all trades
INFO:root:Converting journal trades to DataFrame
INFO:root:Added PA tags to DataFrame. Columns: Index(['type_1_m5', 'type_1_m1', 'type_1_m30', 'type_1_h4', 'type_1_m15',
       'risk_reward_ratio', 'type_1_h1', 'type_1_d', 'type_1_M', 'type_1_w',
       'has_type_1', 'has_type_2', 'has_type_3', 'has_time_frame'],
      dtype='object')
INFO:root:DataFrame columns: Index(['type_1_m5', 'type_1_m1', 'type_1_m30', 'type_1_h4', 'type_1_m15',
       'risk_reward_ratio', 'type_1_h1', 'type_1_d', 'type_1_M', 'type_1_w',
       'has_type_1', 'has_type_2', 'has_type_3', 'has_time_frame'],
      dtype='object')


frozenset({'type_1_m5', 'type_2_m30', 'M', 'type_1_m1', 'type_2_m15', 'type_2_h4', 'm15', 'type_1_m30', 'type_2_M', 'type_2_m1', 'm1', 'type_2_w', 'h4', 'm30', 'type_1_h4', 'type_2_m5', 'type_2_h1', 'type_1_m15', 'd', 'type_1_h1', 'type_1_d', 'm5', 'type_1_M', 'w', 'h1', 'type_2_d', 'type_1_w'})
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [79]:
from src.tradecli import *
from src.utils import *
from src.journal import *

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [80]:
display(full_df)

,type_1_m5,type_1_m1,type_1_m30,type_1_h4,type_1_m15,risk_reward_ratio,type_1_h1,type_1_d,type_1_M,type_1_w,has_type_1,has_type_2,has_type_3,has_time_frame
0,False,True,False,False,False,2.000000,False,False,False,False,True,False,False,False
1,False,False,False,False,False,0.666667,False,False,False,False,False,False,False,False
2,False,False,False,False,False,10.000000,False,False,False,False,False,False,False,False
3,False,False,False,False,False,10.000000,False,False,False,False,False,False,False,False
4,False,False,False,False,False,3.000000,False,True,False,False,True,False,False,False
5,False,False,False,False,False,3.943182,False,False,True,False,True,False,False,False
6,False,False,False,True,False,3.323529,False,False,False,False,True,False,False,False
7,False,True,False,False,False,7.927273,False,False,False,False,True,False,False,False
8,False,False,True,False,False,8.811321,False,False,False,False,True,False,False,False
9,False,False,False,True,False,2.986486,False,False,False,False,True,False,False,False


In [81]:
full_df.dtypes

type_1_m5               bool
type_1_m1               bool
type_1_m30              bool
type_1_h4               bool
type_1_m15              bool
risk_reward_ratio    float64
type_1_h1               bool
type_1_d                bool
type_1_M                bool
type_1_w                bool
has_type_1              bool
has_type_2              bool
has_type_3              bool
has_time_frame          bool
dtype: object

In [82]:
!rm -rf output

In [84]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
j.to_markdown(output_dir='output')

INFO:root:Converting trades to markdown
INFO:root:Files for trades found
INFO:root:Files Found #: 14, Total Trades #: 14max files for trade: 0min files for trade: 0avg files for trade: 0.0
INFO:root:Descriptive Statistics:           0
count  14.0
mean    0.0
std     0.0
min     0.0
25%     0.0
50%     0.0
75%     0.0
max     0.0
INFO:root:Writing index markdown
INFO:root:Calculating simple statistics
INFO:root:Analyzing tag relevance
INFO:root:Finding best single tags and subsets
INFO:root:Finding best single tags
INFO:root:Finding best tag subsets
INFO:root:Finding best tag subsets
INFO:root:Selecting top 10 features using RFE
INFO:root:Dropped 6 rows with NaN in 'outcome' y variable
INFO:root:X features # 41: {'type_3_m1', 'type_3_M', 'htf_poi_ltf_confirmation_False', 'type_2_m30', 'type_1_m5', 'type_1_m1', 'type_3_h4', 'type_3_w', 'htf_poi_ltf_confirmation', 'type_2_m15', 'type_3_m15', 'type_2_h4', 'close_price', 'type_1_m30', 'management_strategy', 'outcome_loss', 'type_3_d', 'type